# Notes
These are just notes I used for constructing the TFP joints

In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
tfd = tfp.distributions
tfb = tfp.bijectors
from tensorflow_probability.python.internal import tensorshape_util

In [2]:
import maxentep

In [3]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]])
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
print(tf.where(bs > 0)[:,1:])


tf.Tensor(
[[0 1]
 [1 2]
 [2 3]
 [3 3]
 [0 1]
 [1 2]
 [2 3]
 [3 3]], shape=(8, 2), dtype=int64)


In [4]:
j = maxentep.recip_norm_mat_dist(np.concatenate((start[np.newaxis,...], start[np.newaxis,...])),
                                 np.concatenate((var[np.newaxis,...], var[np.newaxis,...])))
print(j)

indicies tf.Tensor(
[[[0 1]
  [1 2]
  [2 3]
  [3 3]]

 [[0 1]
  [1 2]
  [2 3]
  [3 3]]], shape=(2, 4, 2), dtype=int32)
bindices tf.Tensor(
[[[0 0]
  [1 1]
  [2 2]
  [3 3]]

 [[0 0]
  [1 1]
  [2 2]
  [3 3]]], shape=(2, 4, 2), dtype=int32)
tf.Tensor(
[[[0]
  [0]
  [0]
  [0]]

 [[1]
  [1]
  [1]
  [1]]], shape=(2, 4, 1), dtype=int32)
tf.Tensor(
[[[0 0 0]
  [0 1 1]
  [0 2 2]
  [0 3 3]]

 [[1 0 0]
  [1 1 1]
  [1 2 2]
  [1 3 3]]], shape=(2, 4, 3), dtype=int32)
tf.Tensor(
[[[0 1]
  [1 2]
  [2 3]
  [3 3]]

 [[0 1]
  [1 2]
  [2 3]
  [3 3]]], shape=(2, 4, 2), dtype=int32) (2, 4, 4)
should get out tf.Tensor(
[[10.  3.  2.  1.]
 [10.  3.  2.  1.]], shape=(2, 4), dtype=float64)
tf.Tensor(
[[0.19488621 0.24884956 0.39727268 0.67659831]
 [0.18456469 0.31191718 0.4107147  0.52669838]], shape=(2, 4), dtype=float64)
tf.Tensor(
[[[0 1]
  [1 2]
  [2 3]
  [3 3]]

 [[0 1]
  [1 2]
  [2 3]
  [3 3]]], shape=(2, 4, 2), dtype=int32)
(2, 4, 4)
tf.Tensor(
[[0.80511379 0.75115044 0.60272732 0.32340169]
 [0.81543531 

In [9]:
j = maxentep.recip_norm_mat_dist(start, var)
print(j)

InvalidArgumentError: Input to reshape is a tensor with 4 values, but the requested shape requires a multiple of 8 [Op:Reshape]

In [8]:
print(j.sample(1))
#j.log_prob(j.sample())

tf.Tensor(
[[[0.08992227 0.38290293 0.4177349  0.55050743]
  [0.06082963 0.29307671 0.81850638 0.63690733]]], shape=(1, 2, 4), dtype=float64)
tf.Tensor(
[[[0 1]
  [1 2]
  [2 3]
  [3 3]]

 [[0 1]
  [1 2]
  [2 3]
  [3 3]]], shape=(2, 4, 2), dtype=int32)
(2, 4, 4)


InvalidArgumentError: Outer dimensions of indices and update must match. Indices shape: [4,2], updates shape:[2,4] [Op:ScatterNd]

In [ ]:
start = np.array([[0.0, 10, 0, 0], [0, 0.0, 3, 0.], [0., 0.0, 0.0, 2.], [0., 0., 0., 1.]]).astype(np.float32)
var = start / 2
bs = np.concatenate((start[np.newaxis,...], start[np.newaxis,...]))
i = tf.reshape(tf.cast(tf.where(bs > 0)[...,1:], tf.int32), (-1, start.shape[0], 2))
v = tf.gather_nd(bs, i, batch_dims=1)
print(i, v)
b = maxentep.Scatter(i, bs.shape[1:])
bs.shape

In [ ]:
b.forward(v)
print(v, b.inverse(b.forward(v)))

In [ ]:
b.inverse_log_det_jacobian(v, 1)

In [ ]:
tf.scatter_nd(i, v, bs.shape)

In [ ]:
tf.map_fn(lambda x: tf.scatter_nd(x[0], x[1], bs.shape[1:]), elems=[i,v],  dtype=tf.float32)

In [ ]:
w = tf.constant(start.astype(np.float32))
w, tf.ones(tf.rank(w))

In [ ]:
i =  tf.keras.Input((1,))
combined = np.stack((start, var))
x  = maxentep.TrainableInputLayer(combined, constraint=maxentep.PositiveMaskedConstraint(combined > 0))(i)

In [ ]:
m = tf.keras.Model(inputs=i, outputs=x)
t = m(tf.constant([1., 1.]))
t

In [ ]:
t[0,0], t[0,1]

In [ ]:
maxentep.recip_norm_mat_dist(t[:1,0], t[:1,1]).sample(5)

In [ ]:
i =  tf.keras.Input((1,))
l = maxentep.recip_norm_mat_layer(i, start, var, 'test')

In [ ]:
m = tf.keras.Model(inputs=i, outputs=l)

In [ ]:
m(tf.constant([1.]))